## Import Libraries

In [1]:
import torch
import math

from torch import optim
from torch import Tensor

from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import numpy as np

## Load the data for Project n°1

In [2]:
import dlc_bci as bci
train_input, train_target = bci.load(root =  './data_bci', train=True, one_khz=True)
train_input = train_input 

print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())
test_input, test_target = bci.load(root =  './data_bci', train=False, one_khz=True)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


## Normalization (by hand)

In [3]:
train_moy = train_input.mean(dim=2).expand(train_input.size(2),train_input.size(0),train_input.size(1)).transpose(0,1).transpose(1,2)
train_std = train_input.std(dim=2).expand(train_input.size(2),train_input.size(0),train_input.size(1)).transpose(0,1).transpose(1,2)
train_input = train_input.sub_(train_moy).div(train_std) #center and reduce

test_moy = test_input.mean(dim=2).expand(test_input.size(2),test_input.size(0),test_input.size(1)).transpose(0,1).transpose(1,2)
test_std = test_input.std(dim=2).expand(test_input.size(2),test_input.size(0),test_input.size(1)).transpose(0,1).transpose(1,2)
test_input = test_input.sub_(test_moy).div(test_std) #center and reduce

## Add the channel dimension

In [4]:
train_input = train_input.view(len(train_input),1,train_input.size(1),-1)
test_input = test_input.view(len(test_input),1,test_input.size(1),-1)

train_input, test_input, train_target, test_target = Variable(train_input), Variable(test_input), Variable(train_target), Variable(test_target)

In [15]:
nb_train_signals = train_input.shape[0]
nb_electrodes = train_input.shape[2]
nb_samples_per_signal = train_input.shape[3] #There was a small error in Thibault's code here, corrected here
print('nb_train_signals = ', nb_train_signals, ', nb_electrodes = ', nb_electrodes, ', nb_samples_per_signal = ', nb_samples_per_signal)
mini_batch_size = 32

nb_train_signals =  316 , nb_electrodes =  28 , nb_samples_per_signal =  500


## Create the architecture of the NN

In [6]:
class Flatten(nn.Module):
    def forward(self,input):
        return input.view(mini_batch_size, -1)

In [7]:
def create_model():
    return nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3),
        nn.ReLU(),
#        nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3),
#        nn.ReLU(),
        nn.MaxPool2d(kernel_size = (2,6)),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3),
        nn.ReLU(),
#        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3),
#        nn.ReLU(),
        nn.MaxPool2d(kernel_size = (2,6)),
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3),
        nn.ReLU(),
#        nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3),
#        nn.ReLU(),
#        nn.MaxPool2d(kernel_size = 2),
        nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3),
        nn.ReLU(),
#        nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3),
#        nn.ReLU(),
        Flatten(),
        nn.Linear(2304,1000), 
        nn.Dropout(0.5),
        nn.ReLU(),
        nn.BatchNorm1d(1000),
        nn.Linear(1000,1000),
        nn.Dropout(0.5),
        nn.ReLU(),
        nn.BatchNorm1d(1000),
        nn.Linear(1000,2),
        nn.ReLU())

In [8]:
#class Thibaults_model(nn.Module):
#    def __init__(self):
#        super(Thibaults_model, self).__init__()
#        self.conv1 = nn.Conv1d(in_channels = nb_electrodes, out_channels = 32, kernel_size = 3) #Length= nb_samples_per_signal-2 = 498
#        self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3) #Length= nb_samples_per_signal-4 = 496
#        self.pool  = nn.MaxPool1d(kernel_size = 2) #Length= (nb_samples_per_signal - 4)/2 = 248
#        self.conv3 = nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3) #(Length= nb_samples_per_signal-4)/2 -2 = 246
#        self.conv4 = nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3) #(Length= nb_samples_per_signal-4)/2 -4 = 244
#        self.conv5 = nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3) #((Length= nb_samples_per_signal-4)/2 -4)/2 -2 = 120
#        self.conv6 = nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3) #((Length= nb_samples_per_signal-4)/2 -4)/2 -4 = 198
#        self.conv7 = nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3) #(((Length= nb_samples_per_signal-4)/2 -4)/2 - 4)/2 -2 = 97
#        self.conv8 = nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3) #(((Length= nb_samples_per_signal-4)/2 -4)/2 - 4)/2  -4= 95
#        self.fc1 = nn.Linear(in_features=14080, out_features=100) #Really not sure, hope so? apparently he recieves 256*32 channels x 55 long signals
#        self.dropout = nn.Dropout(p=0.5)
#        self.fc2 = nn.Linear(in_features=100, out_features=100)
#        self.fc3 = nn.Linear(in_features=100, out_features=2)
#    def forward(self, x):
#        x = F.relu(self.conv1(x)) 
#        x = F.relu(self.conv2(x))
#        x = F.relu(self.conv3(self.pool(x)))
#        x = F.relu(self.conv4(x))
#        x = F.relu(self.conv5(self.pool(x)))
#        x = F.relu(self.conv6(x))
#        x = F.relu(self.conv7(self.pool(x)))
#        x = F.relu(self.conv8(x))
#        x = x.view(mini_batch_size, -1)
#        x = F.relu(self.fc3(F.relu(self.fc2(F.relu(self.fc1(x))))))
#        return x

In [17]:
def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-1) #No idea what those parameters do
    nb_epochs = 50

    for e in range(0, nb_epochs):
        for b in range(math.ceil(train_input.size(0)/mini_batch_size)):
            if ((b+1)*mini_batch_size < train_input.size(0)):
                output = model(train_input.narrow(0, b*mini_batch_size, mini_batch_size))
                loss = criterion(output, train_target.narrow(0, b*mini_batch_size, mini_batch_size))
            else:
                #que du sale mamen
                output = model(train_input.narrow(0, train_input.size(0)-mini_batch_size, mini_batch_size))
                loss = criterion(output, train_target.narrow(0, train_input.size(0)-mini_batch_size, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()

######################################################################

def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(math.ceil(data_input.size(0)/mini_batch_size)):
        if ((b+1)*mini_batch_size < data_input.size(0)):
            output = model(data_input.narrow(0, b*mini_batch_size, mini_batch_size))
            _, predicted_classes = torch.max(output.data, 1)
        else:
            output = model(data_input.narrow(0, data_input.size(0)-mini_batch_size, mini_batch_size))
            _, predicted_classes = torch.max(output.data, 1)
        
        for k in range(0, mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [16]:
model = create_model()

In [18]:
train_model(model, train_input, train_target)
nb_test_errors = compute_nb_errors(model, test_input, test_target)
nb_train_errors = compute_nb_errors(model, train_input, train_target)
print('test error = {:0.2f}% {:d}/{:d} and train error = {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                                                     nb_test_errors, test_input.size(0),
                                                                                     (100 * nb_train_errors) / train_input.size(0),
                                                                                     nb_train_errors, train_input.size(0)))
print('With a batch size of {}' .format(mini_batch_size))

test error = 50.00% 50/100 and train error = 44.62% 141/316
With a batch size of 32
